# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report Code
**_Opening a Pizza Place in Chicago, Illinois USA_**
- Build a dataframe of neighborhoods in Chicago, Illinois by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new pizza place
***
### 1. Import  necessary libraries

In [ ]:
!pip install geocoder

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

from geopy.geocoders import Nominatim 
import geocoder 

import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium 

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into Pandas Dataframe

In [2]:
# send the GET request
data = requests.get("https://simple.wikipedia.org/wiki/Category:Suburbs_of_Chicago,_Illinois").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
ci_df = pd.DataFrame({"Neighborhood": neighborhoodList})

ci_df.head()

,Neighborhood
0,"Addison, Illinois"
1,"Algonquin, Illinois"
2,"Arlington Heights, Illinois"
3,"Aurora, Illinois"
4,"Barrington Hills, Illinois"


In [7]:
# print the number of rows of the dataframe
ci_df.shape
ci_df

,Neighborhood
0,"Addison, Illinois"
1,"Algonquin, Illinois"
2,"Arlington Heights, Illinois"
3,"Aurora, Illinois"
4,"Barrington Hills, Illinois"
5,"Barrington, Illinois"
6,"Bartlett, Illinois"
7,"Bedford Park, Illinois"
8,"Bellwood, Illinois"
9,"Bensenville, Illinois"


## removing uncecessary text, and observation

In [8]:
ci_df['Neighborhood'] = ci_df['Neighborhood'].str.rstrip('Illinois')

In [9]:
ci_df['Neighborhood'] = ci_df['Neighborhood'].str.rstrip('(Chicago)')

In [10]:
ci_df=ci_df.drop(99) #indiana observation
ci_df=ci_df.drop(104)#repeat

In [11]:
#ci_df['Neighborhood'] = ci_df['Neighborhood'].str.rstrip(',')
ci_df['Neighborhood'] = ci_df['Neighborhood'].str.replace(',', '')


In [12]:
ci_df

,Neighborhood
0,Addison
1,Algonquin
2,Arlington Heights
3,Aurora
4,Barrington Hills
5,Barrington
6,Bartlett
7,Bedford Park
8,Bellwood
9,Bensenville


### 3. Get the geographical coordinates

In [13]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chicago, Illinois'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in ci_df["Neighborhood"].tolist() ]

In [15]:
coords

[[41.94711582530031, -87.66075912010247],
 [42.006099467988015, -87.76285438055761],
 [41.926749037603216, -87.64270416822308],
 [41.882829933739, -88.20902030137657],
 [42.11430000000007, -88.18250999999998],
 [41.989599731149426, -87.73532999230926],
 [41.991820000000075, -88.18458999999996],
 [41.76293000000004, -87.79494999999997],
 [41.88193000000007, -87.86695999999995],
 [41.95604000000003, -87.94105999999994],
 [41.80014005543244, -87.60048357704304],
 [41.97403004484906, -87.87110676169897],
 [41.89962000000003, -87.62512999999996],
 [41.64818509534535, -87.63281446187453],
 [41.698210000000074, -88.07963999999998],
 [41.91799000000003, -87.64873999999998],
 [41.86171000000007, -87.86273999999997],
 [41.82295000000005, -87.84058999999996],
 [41.68186504401369, -87.54383025665967],
 [41.884250000000065, -87.63244999999995],
 [41.74914511160509, -87.55646011345749],
 [42.089590000000044, -87.83885999999995],
 [41.73124000000007, -87.55125999999996],
 [41.71490000000006, -87.5316

In [16]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [17]:
# merge the coordinates into the original dataframe
ci_df['Latitude'] = df_coords['Latitude']
ci_df['Longitude'] = df_coords['Longitude']

In [18]:
# check the neighborhoods and the coordinates
print(ci_df.shape)
ci_df

(152, 3)


,Neighborhood,Latitude,Longitude
0,Addison,41.947116,-87.660759
1,Algonquin,42.006099,-87.762854
2,Arlington Heights,41.926749,-87.642704
3,Aurora,41.882830,-88.209020
4,Barrington Hills,42.114300,-88.182510
5,Barrington,41.989600,-87.735330
6,Bartlett,41.991820,-88.184590
7,Bedford Park,41.762930,-87.794950
8,Bellwood,41.881930,-87.866960
9,Bensenville,41.956040,-87.941060


In [19]:
ci_df=ci_df.drop(152) #NA
ci_df=ci_df.drop(153) #NA

In [20]:
# save the DataFrame as CSV file
ci_df.to_csv("ci_df.csv", index=False)

### 4. Create a map of Chicago, Illinois with neighborhoods superimposed on top

In [21]:
# get the coordinates of the city
address = 'Chicago, Illinois'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago Illinois, USA {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago Illinois, USA 41.8755616, -87.6244212.


In [22]:
# create map of chi-town using latitude and longitude values
map_ci = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(ci_df['Latitude'], ci_df['Longitude'], ci_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5).add_to(map_ci)  
    
map_ci

In [23]:
# save the map as HTML file
map_ci.save('map_ci.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [24]:
CLIENT_ID = '11HZ5QGHAEFP0FF2WOUTTLBG42CCTWUUPIHSAEUODDNWH0PJ' # your Foursquare ID
CLIENT_SECRET = 'D0QN0BCYUL2MUXCCVSBWGJUJAOOI54DRCMA3HPXYILLNP4ED' # your Foursquare Secret
VERSION = '20200709' # Foursquare API version

print('d54347n@pace.edu:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

d54347n@pace.edu:
CLIENT_ID: 11HZ5QGHAEFP0FF2WOUTTLBG42CCTWUUPIHSAEUODDNWH0PJ
CLIENT_SECRET:D0QN0BCYUL2MUXCCVSBWGJUJAOOI54DRCMA3HPXYILLNP4ED


**Now, let's get the top 50 venues that are within a radius of 2000 meters.**

In [29]:
radius = 2000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(ci_df['Latitude'], ci_df['Longitude'], ci_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [30]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6687, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Addison,41.947116,-87.660759,Southport Grocery & Cafe,41.946782,-87.664046,Breakfast Spot
1,Addison,41.947116,-87.660759,Jeni’s Splendid Ice Creams,41.948472,-87.657353,Ice Cream Shop
2,Addison,41.947116,-87.660759,Ella Elli,41.945208,-87.663652,American Restaurant
3,Addison,41.947116,-87.660759,Café Tola,41.947474,-87.664002,Empanada Restaurant
4,Addison,41.947116,-87.660759,Guthrie's Tavern,41.947221,-87.661657,Bar


**Let's check how many venues were returned for each neighorhood**

In [31]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Addison,50,50,50,50,50,50
Algonquin,50,50,50,50,50,50
Arlington Heights,50,50,50,50,50,50
Aurora,27,27,27,27,27,27
Barrington,50,50,50,50,50,50
Barrington Hills,4,4,4,4,4,4
Bartlett,35,35,35,35,35,35
Bedford Park,37,37,37,37,37,37
Bellwood,50,50,50,50,50,50


**Let's find out how many unique categories can be curated from all the returned venues**

In [32]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 341 uniques categories.


In [33]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Breakfast Spot', 'Ice Cream Shop', 'American Restaurant',
       'Empanada Restaurant', 'Bar', 'Indie Movie Theater', 'Brewery',
       'Beer Bar', 'Pizza Place', 'Plaza', 'Hotel', 'Sports Bar', 'Café',
       'Music Venue', 'Baseball Stadium', 'Argentinian Restaurant',
       'Theater', 'Taco Place', 'Coffee Shop', 'Cycle Studio',
       'Bookstore', 'Italian Restaurant', 'Shipping Store', 'Donut Shop',
       'Burger Joint', 'Nail Salon', 'Salon / Barbershop',
       'Mexican Restaurant', 'New American Restaurant', 'Thai Restaurant',
       'Fish & Chips Shop', 'Sandwich Place', 'Dive Bar', 'Beer Store',
       'Dessert Shop', 'Grocery Store', 'Sushi Restaurant', 'Multiplex',
       'Trail', 'Sporting Goods Shop', 'Arts & Crafts Store',
       'Antique Shop', 'Furniture / Home Store', 'Bakery',
       'Supplement Shop', 'Nature Preserve', 'Fast Food Restaurant',
       'Bagel Shop', 'Greek Restaurant', 'Tex-Mex Restaurant'],
      dtype=object)

In [34]:
venues_df['VenueCategory'].describe()

count            6687
unique            341
top       Pizza Place
freq              277
Name: VenueCategory, dtype: object

In [35]:
# check if the results contain "Grocery Store"
"Pizza Place" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [36]:
# one hot encoding
ci_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ci_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ci_onehot.columns[-1]] + list(ci_onehot.columns[:-1])
ci_onehot = ci_onehot[fixed_columns]

print(ci_onehot.shape)
ci_onehot.head()

(6687, 342)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bavarian Restaurant,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Car Wash,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Administrative Building,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Socce

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [37]:
ci_grouped = ci_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ci_grouped.shape)
ci_grouped

(150, 342)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bavarian Restaurant,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Trail,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Car Wash,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Administrative Building,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Socce

In [38]:
len(ci_grouped[ci_grouped["Pizza Place"] > 0])

131

**Create a new DataFrame for pizza place data only**

In [39]:
ci_gs = ci_grouped[["Neighborhoods","Pizza Place"]]

In [40]:
ci_gs.head()

,Neighborhoods,Pizza Place
0,Addison,0.040000
1,Algonquin,0.020000
2,Arlington Heights,0.060000
3,Aurora,0.111111
4,Barrington,0.060000


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Chicago IL into 4 clusters.

In [46]:
# set number of clusters
kclusters = 3

kl_clustering = ci_gs.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 0, 2, 1, 2, 1, 2, 2])

In [47]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ci_merged = ci_gs.copy()

# add clustering labels
ci_merged["Cluster Labels"] = kmeans.labels_

In [48]:
ci_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ci_merged.head()

,Neighborhood,Pizza Place,Cluster Labels
0,Addison,0.040000,2
1,Algonquin,0.020000,1
2,Arlington Heights,0.060000,2
3,Aurora,0.111111,0
4,Barrington,0.060000,2


In [49]:
# merge austin_grouped with chicago_data to add latitude/longitude for each neighborhood
ci_merged = ci_merged.join(ci_df.set_index("Neighborhood"), on="Neighborhood")

print(ci_merged.shape)
ci_merged.head() # check the last columns!

(150, 5)


,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
0,Addison,0.040000,2,41.947116,-87.660759
1,Algonquin,0.020000,1,42.006099,-87.762854
2,Arlington Heights,0.060000,2,41.926749,-87.642704
3,Aurora,0.111111,0,41.882830,-88.209020
4,Barrington,0.060000,2,41.989600,-87.735330


In [50]:
# sort the results by Cluster Labels
print(ci_merged.shape)
ci_merged.sort_values(["Cluster Labels"], inplace=True)
ci_merged

(150, 5)


,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
86,Lombard,0.120000,0,41.851700,-87.778714
25,Carpentersville,0.107143,0,42.118230,-88.264520
26,Channahon,0.222222,0,41.437120,-88.220800
118,Phoenix,0.080000,0,41.982465,-87.833116
30,Country Club Hills,0.080000,0,41.915515,-87.824375
31,Countryside,0.080000,0,41.790230,-87.858910
85,Lockport,0.088889,0,41.588960,-88.055910
33,Crestwood,0.100000,0,41.641190,-87.737860
121,River Forest,0.076923,0,41.653846,-87.609655
41,Downers Grove,0.080000,0,41.795340,-88.006510


**Finally, let's visualize the resulting clusters**

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ci_merged['Latitude'], ci_merged['Longitude'], ci_merged['Neighborhood'], ci_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [53]:
ci_merged.loc[ci_merged['Cluster Labels'] == 0]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
86,Lombard,0.120000,0,41.851700,-87.778714
25,Carpentersville,0.107143,0,42.118230,-88.264520
26,Channahon,0.222222,0,41.437120,-88.220800
118,Phoenix,0.080000,0,41.982465,-87.833116
30,Country Club Hills,0.080000,0,41.915515,-87.824375
31,Countryside,0.080000,0,41.790230,-87.858910
85,Lockport,0.088889,0,41.588960,-88.055910
33,Crestwood,0.100000,0,41.641190,-87.737860
121,River Forest,0.076923,0,41.653846,-87.609655
41,Downers Grove,0.080000,0,41.795340,-88.006510


#### Cluster 1

In [54]:
ci_merged.loc[ci_merged['Cluster Labels'] == 1]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
104,Northfield,0.020000,1,41.900794,-87.626255
148,Winnetka,0.023810,1,42.325960,-87.856130
105,Northlake,0.020000,1,41.900794,-87.626255
106,Oak Brook,0.020000,1,41.900794,-87.626255
108,Oak Lawn,0.000000,1,41.474625,-87.573515
91,Medinah,0.020000,1,41.988976,-87.779020
68,Hickory Hills,0.020000,1,41.903823,-87.652744
67,Hazel Crest,0.020000,1,41.963640,-87.652352
65,Harvey,0.020000,1,41.884250,-87.632450
63,Gurnee,0.020000,1,41.884250,-87.632450


#### Cluster 2

In [55]:
ci_merged.loc[ci_merged['Cluster Labels'] == 2]

,Neighborhood,Pizza Place,Cluster Labels,Latitude,Longitude
146,Wheeling,0.060000,2,42.105810,-87.733670
112,Orland Park,0.060000,2,41.995429,-87.814307
129,Sauk Village,0.040000,2,41.907705,-87.633637
115,Palos Heights,0.060000,2,41.727790,-87.715450
144,Westchester,0.060000,2,42.139550,-87.915730
114,Palatine,0.046512,2,41.703780,-87.829230
132,Skokie,0.040000,2,41.758842,-87.595804
127,Rosemont,0.045455,2,41.499832,-87.635580
125,Robbins,0.040000,2,41.996360,-87.662631
124,Riverside,0.060000,2,42.081080,-88.027560


### Webscraping to see Average Rent Price in Chicago

In [67]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
url = 'https://www.rentcafe.com/average-rent-market-trends/us/il/chicago/'
;

''

In [68]:
rent_table = soup.find('table', class_ = 'market-trends market-trends-nhood')

In [100]:
for neighborhood in rent_table.find_all('tbody'):
    rows = neighborhood.find_all('tr')
    for row in rows:
        pl_neighborhood = row.find('th').text.strip()
        pl_money = getattr(row.find('td'), 'text', None)
        print(pl_neighborhood, pl_money)

Neighborhood None
The Island $562
Austin $562
West Pullman $612
Rosemoor $612
Roseland $612
Riverdale $612
Pullman $612
Princeton Park $612
Longwood Manor $612
Fernwood $612
Cottage Grove Heights $612
West Chesterfield $783
Marynook $783
East Chatham $783
Chatham $783
Calumet Heights $783
Burnside $783
Avalon Park $783
Englewood $842
Pilsen $887
Heart of Italy $887
Heart of Chicago $887
West Garfield Park $911
Homan Square $911
Fifth City $911
South Shore $1,021
Jackson Park Highlands $1,021
Little Village $1,060
Douglas Park $1,060
Woodlawn $1,071
Park Manor $1,103
Greater Grand Crossing $1,103
McKinley Park $1,110
West Morgan Park $1,123
West Beverly $1,123
Tally's Corner $1,123
Mount Greenwood Heights $1,123
Mount Greenwood $1,123
Kennedy Park $1,123
Beverly Woods $1,123
Waclawowo $1,124
The Villa $1,124
Saint Ben's $1,124
Jackowo $1,124
Irving Park $1,124
Horner Park $1,124
Avondale $1,124
West Woodlawn $1,138
Uptown Chicago $1,154
Sheridan Park $1,154
Margate Park $1,154
Lakewood 

#### Observations

Beginning with cluster 0 which is portrayed by the red dots, we can see that these neighbourhoods have the highest amount of pizza parlors already establsihed. This means opening a business there could lead to failiure because of the competetition. This cluster is widley populated throughout Chicago except the northern areas. Cluster 1 is represented by the blue dots. It's the biggest cluster and according to foursquare has the least amount of pizza parlors. This could be because Foursquare is missing venue data pertaining to pizza places in these neighborhoods or there really is just low pizza establishments there. I find it hard to believe that the heaviest cluster is the one with the least amount of competetition. Cluster 2 is the smallest and is portrayed with the light green dots. This cluster's presence is mainly on the inner city near Lake Michigan. This cluster could be the most expensive to build a pizza, but has lower competetition compared to cluster 0. Based of this data analysis, finding a neighborhood with no pizza places established in Cluster 1 would be ideal. I also webscraped a data table off rentcafe for average rental prices for Chicago neighborhoods. In case someone wanted to see a particular neighborhood from the clusters I created, they could check and see if that particular neighborhood's rental price is in the printout above. 